# Modern Methods for Text Classification

What technical topics will the reader learn about during this chapter?
- HEADING 1: Classification: Bread and Butter of NLP
- HEADING 2: Classic Machine Learning: LR, DT, RFC, xgboost
- HEADING 3: How to select the best classifiers using ROC curves
- HEADING 4: Ensemble methods: The Intuition
- HEADING 5: Ensemble methods: Programming our own ensembles

Skills learned: For each heading, insert what the reader will learn to DO in this chapter?
- SKILL 1: Using Machine Learning Classifiers
- SKILL 2: scikit-learn
- SKILL 3: Model Evaluation basics (deeper dive in Model Understanding section)
- SKILL 4: Ensemble methods: intuition
- SKILL 5: Writing our own ensemble implementation

In [1]:
import gzip
from urllib.request import urlretrieve
from tqdm import tqdm
import os
# if you are using the fastAI environment, all of these imports work

In [2]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

In [3]:
def get_data(url, filename):
    """
    Download data if the filename does not exist already
    Uses Tqdm to show download progress
    """
    if not os.path.exists(filename):

        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urlretrieve(url, filename, reporthook=t.update_to)

In [4]:
# Let's download some data:
data_url = 'http://files.fast.ai/data/aclImdb.tgz'
get_data(data_url, 'data/imdb.tgz')

Let's extract the files above and see what the directory contains:

In [31]:
for pathroute in os.walk('data\\imdb\\aclImdb'):
    next_path = pathroute[1]
    for stop in next_path:
        print(stop)

test
train
all
neg
pos
all
neg
pos
unsup


This really badly written utility tells us that there are atleast two folders: `train` and `test`. Each of these folders in turn has atleast 3 folders:
```bash
Test
|- all
|- neg
|- pos
```
and

```bash
Train
|- all
|- neg
|- pos
|- unsup
```

The pos and neg folders contain reviews which are positive and negative respectively. The `unsup` folder stands for unsupervised. They are useful for building language models, specially for Deep Learning. We will not use that here. Similarly, all is redundant because these reviews are repeated in pos and neg folders. 

In [33]:
import spacy
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer